<a href="https://colab.research.google.com/github/007vmachuca/Clase2PF/blob/master/BookExercises/Chapter14_Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 9.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633977 sha256=21b2e3c5ef0a13cc47bfcd8a84f5ca81403aa00cd835ce5a0d34f3cb51cbd46d
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import heapq
from collections import defaultdict
import pandas as pd
import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

In [ ]:
# Load and preprocess data set
fp_df = pd.read_csv("/content/drive/MyDrive/Galit_Shmueli_codes/BookDataSets/Faceplate.csv")
fp_df.set_index("Transaction",
                inplace=True)
print(fp_df)

             Red  White  Blue  Orange  Green  Yellow
Transaction                                         
1              1      1     0       0      1       0
2              0      1     0       1      0       0
3              0      1     1       0      0       0
4              1      1     0       1      0       0
5              1      0     1       0      0       0
6              0      1     1       0      0       0
7              1      0     1       0      0       0
8              1      1     1       0      1       0
9              1      1     1       0      0       0
10             0      0     0       0      0       1


In [ ]:
# create frequent itemsets
itemsets = apriori(fp_df,
                   min_support=0.2,
                   use_colnames=True)
itemsets.head()

,support,itemsets
0,0.6,(Red)
1,0.7,(White)
2,0.6,(Blue)
3,0.2,(Orange)
4,0.2,(Green)


In [ ]:
# convert into rules
rules = association_rules(itemsets,
                          metric="confidence",
                          min_threshold=0.5)
rules.sort_values(by=["lift"],
                  ascending=False).head(6)
print(rules.sort_values(by=["lift"],
                        ascending=False).drop(columns=["antecedent support","consequent support","conviction"]).head(6))
                  

       antecedents   consequents  support  confidence      lift  leverage
12    (Red, White)       (Green)      0.2         0.5  2.500000      0.12
15         (Green)  (Red, White)      0.2         1.0  2.500000      0.12
4          (Green)         (Red)      0.2         1.0  1.666667      0.08
14  (White, Green)         (Red)      0.2         1.0  1.666667      0.08
7         (Orange)       (White)      0.2         1.0  1.428571      0.06
8          (Green)       (White)      0.2         1.0  1.428571      0.06


In [ ]:
# Rules for similar book purchases

In [ ]:
# load dataset
all_books_df = pd.read_csv("/content/drive/MyDrive/Galit_Shmueli_codes/BookDataSets/CharlesBookClub.csv")


In [ ]:
all_books_df.shape, all_books_df.columns

((4000, 24),
 Index(['Seq#', 'ID#', 'Gender', 'M', 'R', 'F', 'FirstPurch', 'ChildBks',
        'YouthBks', 'CookBks', 'DoItYBks', 'RefBks', 'ArtBks', 'GeogBks',
        'ItalCook', 'ItalAtlas', 'ItalArt', 'Florence', 'Related Purchase',
        'Mcode', 'Rcode', 'Fcode', 'Yes_Florence', 'No_Florence'],
       dtype='object'))

In [ ]:

# create the binary incident matrix
ignore = ['Seq#', 'ID#', 'Gender', 'M', 'R', 'F', 'FirstPurch','Related Purchase','Mcode', 'Rcode', 'Fcode', 'Yes_Florence', 'No_Florence']
count_books = all_books_df.drop(columns=ignore)
count_books.head()

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt,Florence
0,0,1,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,2,1,2,0,1,0,1,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
count_books[count_books>0] = 1
# create frequent items and rules
itemsets = apriori(count_books, 
                   min_support = 200/4000,
                   use_colnames = True)
itemsets


,support,itemsets
0,0.39400,(ChildBks)
1,0.23825,(YouthBks)
2,0.41550,(CookBks)
3,0.25475,(DoItYBks)
4,0.20475,(RefBks)
...,...,...
56,0.05525,"(YouthBks, ChildBks, CookBks, RefBks)"
57,0.06325,"(YouthBks, ChildBks, CookBks, GeogBks)"
58,0.06125,"(ChildBks, DoItYBks, CookBks, RefBks)"
59,0.06050,"(ChildBks, DoItYBks, CookBks, GeogBks)"


In [ ]:
rules = association_rules(itemsets,
                          metric="confidence",
                          min_threshold=0.5)
# Display 25 rules with highest lisf
rules.sort_values(by=["lift"], ascending=False).drop(columns=["antecedent support","consequent support","conviction"]).head(25)

,antecedents,consequents,support,confidence,lift,leverage
64,"(YouthBks, RefBks)","(ChildBks, CookBks)",0.05525,0.680000,2.809917,0.035588
73,"(DoItYBks, RefBks)","(ChildBks, CookBks)",0.06125,0.662162,2.736207,0.038865
60,"(YouthBks, DoItYBks)","(ChildBks, CookBks)",0.06700,0.648910,2.681448,0.042014
80,"(GeogBks, RefBks)","(ChildBks, CookBks)",0.05025,0.614679,2.539995,0.030467
69,"(YouthBks, GeogBks)","(ChildBks, CookBks)",0.06325,0.605263,2.501087,0.037961
77,"(GeogBks, DoItYBks)","(ChildBks, CookBks)",0.06050,0.599010,2.475248,0.036058
68,"(ChildBks, CookBks, GeogBks)",(YouthBks),0.06325,0.577626,2.424452,0.037162
71,"(ChildBks, CookBks, RefBks)",(DoItYBks),0.06125,0.591787,2.323013,0.034883
49,"(DoItYBks, GeogBks)",(YouthBks),0.05450,0.539604,2.264864,0.030437
63,"(ChildBks, CookBks, RefBks)",(YouthBks),0.05525,0.533816,2.240573,0.030591


In [ ]:
rules.iloc[22:25]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
22,"(ChildBks, DoItYBks)",(CookBks),0.16150,0.41550,0.12775,0.791022,1.903783,0.060647,2.796941
23,"(ChildBks, CookBks)",(DoItYBks),0.24200,0.25475,0.12775,0.527893,2.072198,0.066101,1.578560
24,"(CookBks, DoItYBks)",(ChildBks),0.16875,0.39400,0.12775,0.757037,1.921414,0.061262,2.494207


# Example 3: Netflix Prize Contest

In [ ]:
import random
random.seed(0)
nratings = 5000
randomData = pd.DataFrame({
    "itemID": [random.randint(0,99) for _ in range(nratings)],
    "userID": [random.randint(0,999) for _ in range(nratings)],
    "rating": [random.randint(1,5) for _ in range(nratings)]
})

def get_top_n(predictions, n=10):
  # First map the preditions to each user
  byUser = defaultdict(list)
  for p in predictions:
    byUser[p.uid].append(p)
  
  # For each user, reduce predictions to top-n
  for uid, userPredictions in byUser.items():
    byUser[uid] = heapq.nlargest(n, userPredictions, key =lambda p:p.est)
  
  return byUser
  

In [ ]:
# Convert the data set into the format required by the surprise package
# The columns must correspond to user id, item id, and ratings (in that order)
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(randomData[["userID","itemID","rating"]],reader)

# Split into training and test set
trainset, testset = train_test_split(data,
                                     test_size=.25,
                                     random_state = 1)



In [ ]:
# User-based filtering
# compute cosine similarity between users
sim_options = {"name":"cosine", "user_based":True}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# predict ratings for all pairs (u,i) that are NOT in the training set.
predictions = algo.test(testset)

# Print the recommended items for each user
top_n = get_top_n(predictions, n=4)
for uid, user_ratings in list(top_n.items())[:5]:
  print("User {}".format(uid))
  for prediction in user_ratings:
    print("   Item {0.iid} ({0.est:.2f})".format(prediction),end="")
    print()


Computing the cosine similarity matrix...
Done computing similarity matrix.
User 6
   Item 6 (5.00)
   Item 77 (2.50)
   Item 60 (1.00)
User 222
   Item 77 (3.50)
   Item 75 (2.78)
User 424
   Item 14 (3.50)
   Item 45 (3.10)
   Item 54 (2.34)
User 87
   Item 27 (3.00)
   Item 54 (3.00)
   Item 82 (3.00)
   Item 32 (1.00)
User 121
   Item 98 (3.48)
   Item 32 (2.83)


In [ ]:
# Rebuild model using the full dataset
trainset = data.build_full_trainset()
sim_options = {"name":"cosine", "user_based":False}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# Predict rating for user 383 and item 7
algo.predict(383,7)

Computing the cosine similarity matrix...
Done computing similarity matrix.


Prediction(uid=383, iid=7, r_ui=None, est=2.3661840936304324, details={'actual_k': 4, 'was_impossible': False})